### Домашнее задание 1 - 10 баллов

1. Загрузите набор данных lenta-ru-news с помощью библиотеки Corus для задачи классификации текстов по топикам (пригодятся атрибуты title, text, topic)- **1 балл**
2. Подготовьте данные к обучению: - **3 балла**
    - Предобработайте данные: реализуйте оптимальную, на ваш взгляд, предобработку текстов (нормализация, очистка, стемминг/лемматизация и т.п.) и таргета.
    - **hint**: для ускорения обработки  и обучения можно ограничиться не всем датасетом, а его репрезентативной частью, например, размера 100_000.
    - Кратко опишите пайплайн, на котором остановились, и почему.
    - Разделите датасет на обучающую, валидационную и тестовую выборки со стратификацией в пропорции 60/20/20. В качестве целевой переменной используйте атрибут `topic`
3. Замерьте базовое качество с любым dummy-бейзлайном - **0.5 балла**
4. Обучите модель `sklearn.linear_model.LogisticRegression` с двумя вариантами векторизации: **2 балла**
  - `sklearn.feature_extraction.text.CountVectorizer`
  - `sklearn.feature_extraction.text.TfidfVectorizer`
5. Попробуйте улучшить качество, подобрав оптимальные гиперпараметры трансформаций и модели на кросс-валидации **1 балл**
6. Оцените качество лучшего пайплайна на отложенной выборке - **0.5 балла**

**Общее**

- Принимаемые решения обоснованы (почему выбрана определенная архитектура/гиперпараметр/оптимизатор/преобразование и т.п.) - **1 балл**
- Обеспечена воспроизводимость решения: зафиксированы random_state, ноутбук воспроизводится от начала до конца без ошибок - **1 балл**

In [48]:
import pandas as pd
import numpy as np
import re
import itertools
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
import warnings
from tqdm import tqdm
from joblib import Parallel, delayed
import optuna

warnings.filterwarnings('ignore')

tqdm.pandas()

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ivawi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ivawi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ivawi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# 1. Загрузка данных

In [ ]:
# !curl -L -o data/lenta-ru-news.csv.gz https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [ ]:
limit = 100000

# не увидела смысла использовать load_lenta из corus - все равно потом складывать все в pd.DataFrame,
# к тому же прямая загрузка быстрее работала
# records = load_lenta('data/lenta-ru-news.csv.gz')
records = pd.read_csv('data/lenta-ru-news.csv.gz', compression='gzip')
df = pd.DataFrame(records)[['title', 'text', 'topic']][:limit]

df.head()

,title,text,topic
0,Названы регионы России с самой высокой смертно...,Вице-премьер по социальным вопросам Татьяна Го...,Россия
1,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт
2,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия
3,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир
4,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир


In [38]:
df.topic.value_counts()

topic
Россия               15151
Мир                  14421
Спорт                10045
Экономика             7682
Интернет и СМИ        6935
Силовые структуры     6925
Бывший СССР           6810
Культура              6578
Наука и техника       5645
Из жизни              4903
Ценности              4480
Дом                   3408
Путешествия           3223
Бизнес                1993
69-я параллель         815
Крым                   661
Культпросвет           307
Оружие                   1
Name: count, dtype: int64

# 2. Подготовка данных

In [ ]:
STOP_RU = list(stopwords.words('russian'))


def preprocess_text(text, stopwords=STOP_RU):
    tokens = re.sub(r'[^а-яё]', ' ', text.lower()).split()
    tokens = [word for word in tokens if word not in stopwords]
    # lemmatized = [mystem.lemmatize(word)[0] for word in tokens]
    return ' '.join(tokens)

In [7]:
df['processed_text'] = Parallel(n_jobs=-1)(delayed(preprocess_text)(t) for t in tqdm(df['text']))

100%|██████████| 100000/100000 [00:02<00:00, 41028.48it/s]


In [ ]:
batch_size = 1000
texts = list(df['processed_text'])
text_batch = [texts[i : i + batch_size] for i in range(0, len(texts), batch_size)]

m = Mystem()


def lemmatize(text):
    merged_text = '|'.join(text)

    doc = []
    res = []

    for t in m.lemmatize(merged_text):
        if t != '|':
            doc.append(t)
        else:
            res.append(''.join(doc))
            doc = []
    res.append(''.join(doc))
    return res


processed_texts = list(itertools.chain(*Parallel(n_jobs=-1)(delayed(lemmatize)(t) for t in tqdm(text_batch))))
df['lemmatized_text'] = processed_texts

100%|██████████| 100/100 [02:19<00:00,  1.40s/it]


In [34]:
df.head()

,title,text,topic,processed_text,lemmatized_text
0,Названы регионы России с самой высокой смертно...,Вице-премьер по социальным вопросам Татьяна Го...,Россия,вице премьер социальным вопросам татьяна голик...,вица премьер социальный вопрос татьяна голиков...
1,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,австрийские правоохранительные органы представ...,австрийский правоохранительный орган представл...
2,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,сотрудники социальной сети проанализировали по...,сотрудник социальный сеть проанализировать пос...
3,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,начала расследования российского вмешательства...,начинать расследование российский вмешательств...
4,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,хакерская группировка опубликовала новые докум...,хакерский группировка опубликовывать новый док...


In [36]:
df.to_csv('data/clean_lenta-ru-news.csv', index=False)

Остановилась на самом классическом варианте: привела к нижнему регистру, убрала лишние символы, токенизировала по словам, далее использовала лемматизацию. Остановилась на ней, так как у нас нет ограничения на время обработки датасета и можно распараллелить, если не устраивает скорость. Лемматизация просто более сложная процедура чем стемминг и теряет меньше смыслов в предложении.

In [39]:
# у меня получилось так, что topic Оружие встретился всего 1 раз, поэтому пришлось его удалить из-за невозможности стратификации
df = df[df['topic'] != 'Оружие']

df = df.dropna()
X = df['lemmatized_text']
y = df['topic']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

In [40]:
X_train.shape, X_val.shape, X_test.shape

((59989,), (19996,), (19997,))

# 3. Dummy

In [ ]:
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(X_train, y_train)
dummy_pred = dummy.predict(X_val)
print(f'Dummy accuracy: {accuracy_score(y_val, dummy_pred):.4f}')

Dummy accuracy: 0.1515


# 4. Logistic Regression

In [ ]:
vectorizers = {'CountVectorizer': CountVectorizer(), 'TfidfVectorizer': TfidfVectorizer()}

for name, vectorizer in vectorizers.items():
    pipeline = Pipeline(
        [('vectorizer', vectorizer), ('classifier', LogisticRegression(max_iter=1000, random_state=42))]
    )

    pipeline.fit(X_train, y_train)
    val_pred = pipeline.predict(X_val)
    accuracy = accuracy_score(y_val, val_pred)
    print(f'{name} accuracy: {accuracy:.4f}')

CountVectorizer accuracy: 0.8380
TfidfVectorizer accuracy: 0.8342


# 5. (Op)Tuning :)

In [ ]:
# использовала optuna, так у нее под капотом метод TPE, который работает быстрее и точнее того же простого grid search
# TPE - метод байесовской оптимизации, который на каждом шаге пытается предсказать, какие параметры дадут лучший результат,
# строя 2 вероятностные модели. Это к пункту про то, почему были выбраны определенные параметры трансформаций и модели
# - потому что они дают лучший результат (хоть и не намного)
def objective(trial):
    vectorizer_type = trial.suggest_categorical('vectorizer', ['CountVectorizer', 'TfidfVectorizer'])
    # ngram_range = trial.suggest_categorical('ngram_range', [(1, 1), (1, 2)]) # по памяти не вывозит
    max_df = trial.suggest_float('max_df', 0.8, 0.99)
    min_df = trial.suggest_int('min_df', 2, 15)
    C = trial.suggest_loguniform('C', 0.01, 10.0)
    max_iter = trial.suggest_int('max_iter', 300, 1000)
    solver = trial.suggest_categorical('solver', ['liblinear', 'lbfgs'])
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2', None])
    use_scaler = trial.suggest_categorical('use_scaler', [True, False])

    invalid_combinations = {
        'liblinear': [None],
        'lbfgs': ['l1'],
    }

    if penalty in invalid_combinations.get(solver, []):
        return float('-inf')
        # возвращаем плохой результат, чтобы Optuna отбросила эту комбинацию
        # тк есть solvers, которые не работают с определенными penalty

    if vectorizer_type == 'CountVectorizer':
        vectorizer = CountVectorizer(max_df=max_df, min_df=min_df)
    else:
        vectorizer = TfidfVectorizer(max_df=max_df, min_df=min_df)

    if use_scaler:
        pipeline = Pipeline(
            [
                ('vectorizer', vectorizer),
                ('scaler', StandardScaler(with_mean=False)),
                (
                    'classifier',
                    LogisticRegression(C=C, random_state=42, max_iter=max_iter, solver=solver, penalty=penalty),
                ),
            ]
        )
    else:
        pipeline = Pipeline(
            [
                ('vectorizer', vectorizer),
                (
                    'classifier',
                    LogisticRegression(C=C, random_state=42, max_iter=max_iter, solver=solver, penalty=penalty),
                ),
            ]
        )

    scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)
    return np.mean(scores)


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

[I 2025-02-25 19:33:23,671] A new study created in memory with name: no-name-3a6f2cfe-6ffe-49c0-86b0-199ab8f418b6
[I 2025-02-25 19:34:03,092] Trial 0 finished with value: 0.8213839979970544 and parameters: {'vectorizer': 'TfidfVectorizer', 'max_df': 0.8351948636043404, 'min_df': 15, 'C': 1.2238156722163405, 'max_iter': 746, 'solver': 'lbfgs', 'penalty': None, 'use_scaler': False}. Best is trial 0 with value: 0.8213839979970544.
[I 2025-02-25 19:34:12,919] Trial 1 finished with value: 0.8172331867208621 and parameters: {'vectorizer': 'TfidfVectorizer', 'max_df': 0.9077070665925416, 'min_df': 9, 'C': 1.0609593278868934, 'max_iter': 785, 'solver': 'lbfgs', 'penalty': 'l2', 'use_scaler': True}. Best is trial 0 with value: 0.8213839979970544.
[I 2025-02-25 19:34:12,921] Trial 2 finished with value: -inf and parameters: {'vectorizer': 'TfidfVectorizer', 'max_df': 0.9226728970836465, 'min_df': 9, 'C': 0.04356038495172463, 'max_iter': 712, 'solver': 'liblinear', 'penalty': None, 'use_scaler': 

In [70]:
best_params = study.best_params
print(best_params)

{'vectorizer': 'CountVectorizer', 'max_df': 0.8413527045302485, 'min_df': 2, 'C': 0.04900401112747474, 'max_iter': 365, 'solver': 'liblinear', 'penalty': 'l2', 'use_scaler': False}


# 6. Final accuracy

In [ ]:
from sklearn.metrics import classification_report

vectorizer = CountVectorizer if best_params['vectorizer'] == 'CountVectorizer' else TfidfVectorizer

if best_params['use_scaler']:
    best_pipeline = Pipeline(
        [
            ('vectorizer', vectorizer(max_df=best_params['max_df'], min_df=best_params['min_df'])),
            ('scaler', StandardScaler(with_mean=False)),
            (
                'classifier',
                LogisticRegression(
                    C=best_params['C'],
                    random_state=42,
                    max_iter=best_params['max_iter'],
                    solver=best_params['solver'],
                    penalty=best_params['penalty'],
                ),
            ),
        ]
    )
else:
    best_pipeline = Pipeline(
        [
            ('vectorizer', vectorizer(max_df=best_params['max_df'], min_df=best_params['min_df'])),
            (
                'classifier',
                LogisticRegression(
                    C=best_params['C'],
                    random_state=42,
                    max_iter=best_params['max_iter'],
                    solver=best_params['solver'],
                    penalty=best_params['penalty'],
                ),
            ),
        ]
    )

best_pipeline.fit(X_train, y_train)
y_pred = best_pipeline.predict(X_test)

print('Best hyperparameters:', best_params)
print('\nPerformance on test set:')
print(classification_report(y_test, y_pred))

Best hyperparameters: {'vectorizer': 'CountVectorizer', 'max_df': 0.8413527045302485, 'min_df': 2, 'C': 0.04900401112747474, 'max_iter': 365, 'solver': 'liblinear', 'penalty': 'l2', 'use_scaler': False}

Performance on test set:
                   precision    recall  f1-score   support

   69-я параллель       0.89      0.65      0.75       163
           Бизнес       0.64      0.49      0.56       398
      Бывший СССР       0.86      0.85      0.85      1362
              Дом       0.88      0.84      0.86       681
         Из жизни       0.81      0.78      0.79       981
   Интернет и СМИ       0.82      0.80      0.81      1387
             Крым       0.78      0.61      0.69       132
    Культпросвет        0.71      0.33      0.45        61
         Культура       0.87      0.91      0.89      1316
              Мир       0.83      0.89      0.86      2885
  Наука и техника       0.87      0.88      0.88      1129
      Путешествия       0.87      0.82      0.84       644
   